In [1]:
import torch
from torch.functional import F
import time
import numpy as np

In [2]:
user = 3176
target = 1024
micro = 128
B, L, H, D = 1, user+target, 3, 256

# seq len: 3171 user length + 200 target length. ---> 3171 user length + 4096 target length

q = torch.randn(B, L, H, D, dtype=torch.half, device="cuda")
k = torch.randn(B, L, H, D, dtype=torch.half, device="cuda")
v = torch.randn(B, L, H, D, dtype=torch.half, device="cuda")
## Experiment 1
rab = torch.ones(B,1,L,L,dtype=torch.half, device="cuda")
def construct_mask():
    user_matrix_len = 23 + 6 + 2 + 5 # 5 for padding 
    rt_seq_max_len = 100
    rt_seq_length = 85
    shopping_cart_seq_max_len = 40
    shopping_cart_seq_length = 30

    lifelong_seq_max_len = 3000
    long_seq_length = 1024
    target_seq_max_len = target
    input_length = user_matrix_len + rt_seq_max_len + shopping_cart_seq_max_len + lifelong_seq_max_len + target_seq_max_len

    device = "cuda"
    dtype = torch.half

    # 计算attention mask
    user_mask = torch.ones([1, input_length, user_matrix_len], dtype=torch.bool, device=device)

    rt_attn_mask = torch.zeros([1, input_length, rt_seq_max_len], dtype=torch.bool, device=device)
    # rt_autoregressive_mask = torch.triu(
    #     torch.ones([1, rt_seq_length, rt_seq_length], dtype=torch.bool, device=device))
    # rt_attn_mask[:, user_matrix_len:user_matrix_len + rt_seq_length, :rt_seq_length] = rt_autoregressive_mask
    rt_autoregressive_mask = torch.arange(rt_seq_max_len, device=device)
    rt_autoregressive_mask = rt_autoregressive_mask.view(1, rt_seq_max_len, 1) <= rt_autoregressive_mask.view(1, 1, rt_seq_max_len)
    rt_valid_mask = (
            torch.arange(rt_seq_max_len, device=device) < rt_seq_length).view(1, 1, rt_seq_max_len).expand(1, rt_seq_max_len, rt_seq_max_len)
    rt_attn_mask[:, user_matrix_len:user_matrix_len+rt_seq_max_len, :] = torch.logical_and(rt_autoregressive_mask, rt_valid_mask)

    rt_target_attn_mask = (
            torch.arange(rt_seq_max_len, device=device) < rt_seq_length).view(1, 1, rt_seq_max_len).expand(1, target_seq_max_len, rt_seq_max_len)
    rt_attn_mask[:, -target_seq_max_len:, :] = rt_target_attn_mask

    shopping_attn_mask = (
            torch.arange(shopping_cart_seq_max_len, device=device) < shopping_cart_seq_length).view(1, 1, shopping_cart_seq_max_len).expand(1, input_length, shopping_cart_seq_max_len)

    lifelong_attn_mask = (
            torch.arange(lifelong_seq_max_len, device=device) < long_seq_length).view(1, 1, lifelong_seq_max_len).expand(1, input_length, lifelong_seq_max_len)

    target_attn_mask = torch.zeros([1, input_length, target_seq_max_len], dtype=torch.bool, device=device)
    target_attn_mask[:, -target_seq_max_len:, :] = torch.diag(torch.ones(target_seq_max_len, dtype=torch.bool, device=device)).unsqueeze(dim=0)

    invalid_attn_mask = torch.cat([user_mask, rt_attn_mask, shopping_attn_mask, lifelong_attn_mask, target_attn_mask], dim=-1)

    return invalid_attn_mask

mask = construct_mask().to(torch.half)

def hstu_attention_v1(
        q: torch.Tensor,
        k: torch.Tensor,
        v: torch.Tensor,
        num_heads: int,
        attention_dim: int,
        linear_dim: int,
        avg_factor: int,
        invalid_attn_mask: torch.Tensor
) -> torch.Tensor:
    b, n, _ = q.size()
    m = k.size(1)

    #print("verbose {} {} {}".format(b, n, m)) # 1 3371 3371
    qk_attn = torch.einsum(
        "bnhd,bmhd->bhnm",
        q.reshape(b, n, num_heads, attention_dim),
        k.reshape(b, m, num_heads, attention_dim),
    )
    qk_attn = qk_attn + rab.repeat(1,b,1,1)
    # print(qk_attn.shape)            # torch.Size([1, 2, 3371, 3371])
    qk_attn = F.silu(qk_attn) / avg_factor
    # print(qk_attn.shape)            # torch.Size([1, 2, 3371, 3371])
    qk_attn = qk_attn * invalid_attn_mask.unsqueeze(dim=1)
    # print(qk_attn.shape)            # torch.Size([1, 2, 3371, 3371])
    attn_output = torch.einsum(
        "bhnm,bmhd->bnhd",
        qk_attn,
        v.reshape(b, m, num_heads, linear_dim)
    ).reshape(b, n, num_heads * linear_dim)
    # print(attn_output.shape)        # torch.Size([1, 3371, 512])
    return attn_output


In [3]:
def test_hstu_attention_v2():
    """ pybind test nv hstu kernel
    """
    from hstu import run_hstu
    # from hstu_attn import hstu_attn_varlen_func
    
    cu_seqlens_q = torch.tensor([0, L], dtype=torch.int32, device="cuda")
    cu_seqlens_k = torch.tensor([0, L], dtype=torch.int32, device="cuda")
    
    user_length = torch.tensor([user],device = "cuda")
    target_length = torch.tensor([target],device = "cuda")
    micro_bs = torch.tensor([micro], device="cuda")


    res = run_hstu(q.reshape(B * L, H, D).half(),
            k.reshape(B * L, H, D).half(),
            v.reshape(B * L, H, D).half(),cu_seqlens_q,cu_seqlens_k,user+target,
            user+target,user_length,micro_bs,target_length,mask)
    return res

In [4]:
def test_hstu_attention_v3():
    """ pybind test nv hstu kernel
    """
    from hstu import run_mask
    # from hstu_attn import hstu_attn_varlen_func
    
    cu_seqlens_q = torch.tensor([0, L], dtype=torch.int32, device="cuda")
    cu_seqlens_k = torch.tensor([0, L], dtype=torch.int32, device="cuda")


    res = run_mask(q.reshape(B * L, H, D).half(),
            k.reshape(B * L, H, D).half(),
            v.reshape(B * L, H, D).half(),cu_seqlens_q,cu_seqlens_k,user+target,user+target,mask)
    return res

In [10]:
def test_mask_rab():
    from hstu import run_mask_rab
    # from hstu_attn import hstu_attn_varlen_func
    
    cu_seqlens_q = torch.tensor([0, L], dtype=torch.int32, device="cuda")
    cu_seqlens_k = torch.tensor([0, L], dtype=torch.int32, device="cuda")


    res = run_mask_rab(q.reshape(B * L, H, D).half(),
            k.reshape(B * L, H, D).half(),
            v.reshape(B * L, H, D).half(),cu_seqlens_q,cu_seqlens_k,user+target,user+target,mask,rab)
    return res

In [11]:
def test_diff():
    o1 = hstu_attention_v1(q.reshape(B, L, H * D),
        k.reshape(B, L, H * D),
        v.reshape(B, L, H * D),
        H, 
        D,
        D,
        L,
        mask).reshape(-1).to(device='cpu')
    o2 = test_hstu_attention_v2().reshape(-1).to(device='cpu')
    o3 = test_hstu_attention_v3().reshape(-1).to(device='cpu')
    o4 = test_mask_rab().reshape(-1).to(device='cpu')
    print(o1)
    print(o2)
    print(o3)
    print(o4)

    difference = o1.numpy() - o4.numpy()

    print(f"50 percentile : {np.percentile(difference, 50)}")
    print(f"80 percentile : {np.percentile(difference, 80)}")
    print(f"90 percentile : {np.percentile(difference, 90)}")
    print(f"99 percentile : {np.percentile(difference, 99)}")
    pass

    return o1,o3

In [12]:
o1,o2 = test_diff()

tensor([ 0.0812,  0.0423, -0.0554,  ...,  0.0789, -0.0472, -0.0142],
       dtype=torch.float16)
tensor([ 0.0773,  0.0404, -0.0517,  ...,  0.0757, -0.0450, -0.0097],
       dtype=torch.float16)
tensor([ 0.0773,  0.0404, -0.0517,  ...,  0.0757, -0.0450, -0.0097],
       dtype=torch.float16)
tensor([ 0.0812,  0.0423, -0.0554,  ...,  0.0788, -0.0472, -0.0142],
       dtype=torch.float16)
50 percentile : 0.0
80 percentile : 1.52587890625e-05
90 percentile : 3.0517578125e-05
99 percentile : 0.0001220703125
